In [ ]:
import pandas as pd

# === 1) Zensus-Daten einlesen und vorbereiten ===
zensus_path = r"C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\INKAR\Absolutzahlen.csv"
zensus_df = pd.read_csv(
    zensus_path,
    sep=';',
    encoding='utf-8',
    quotechar='"',
    dtype={'Kennziffer': str}
)

# Nicht benötigte Spalten entfernen
zensus_df.drop(columns=['Raumeinheit', 'Aggregat'], errors='ignore', inplace=True)

# Erste Zeile enthält Jahresangaben, als neue Spaltennamen verwenden
year_row = zensus_df.iloc[0]
new_columns = [
    f"{col}_{year_row[col]}" if str(year_row[col]).strip().isdigit() else col
    for col in zensus_df.columns
]
zensus_df.columns = new_columns

# Erste Zeile (mit Jahrwerten) löschen
zensus_df = zensus_df.iloc[1:].reset_index(drop=True)

# === 2) KRS-Übersicht einlesen ===
krs_path = r"C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\Verwaltungsgebiete\KRS_Uebersicht.csv"
krs_df = pd.read_csv(
    krs_path,
    sep=',',
    encoding='utf-8',
    quotechar='"',
    dtype={'AGS': str}
)

# === 3) Merge vorbereiten und durchführen ===
zensus_df['Kennziffer'] = zensus_df['Kennziffer'].astype(str)

# Gemeinsame Spalten (außer Join-Spalten) entfernen, um Duplikate zu vermeiden
shared_cols = [col for col in krs_df.columns if col in zensus_df.columns and col != 'Kennziffer']
zensus_clean = zensus_df.drop(columns=shared_cols)

# Merge über AGS (KRS) und Kennziffer (Zensus)
merged_df = pd.merge(
    zensus_clean,
    krs_df,
    left_on='Kennziffer',
    right_on='AGS',
    how='left',
    validate='one_to_many'
)

# 'Kennziffer' entfernen und Spaltenreihenfolge anpassen
merged_df.drop(columns='Kennziffer', inplace=True)
krs_cols = krs_df.columns.tolist()
other_cols = [col for col in merged_df.columns if col not in krs_cols]
merged_df = merged_df[krs_cols + other_cols]

# === 4) Ausgabe prüfen ===
print(merged_df.head())


In [94]:
import pandas as pd
import os
from glob import glob

# Funktion zum Bereinigen einer Zensus-Datei
def clean_zensus(zensus_path, krs_df):
    # Zensus-Daten einlesen und vorbereiten
    zensus_df = pd.read_csv(
        zensus_path,
        sep=';',
        encoding='utf-8',
        quotechar='"',
        dtype={'Kennziffer': str}
    )

    # Nicht benötigte Spalten entfernen
    zensus_df.drop(columns=['Raumeinheit', 'Aggregat'], errors='ignore', inplace=True)

    # Erste Zeile enthält Jahresangaben, als neue Spaltennamen verwenden
    year_row = zensus_df.iloc[0]
    new_columns = [
        f"{col}_{year_row[col]}" if str(year_row[col]).strip().isdigit() else col
        for col in zensus_df.columns
    ]
    zensus_df.columns = new_columns

    # Erste Zeile (mit Jahrwerten) löschen
    zensus_df = zensus_df.iloc[1:].reset_index(drop=True)

    # Merge vorbereiten und durchführen
    zensus_df['Kennziffer'] = zensus_df['Kennziffer'].astype(str)

    # Gemeinsame Spalten (außer Join-Spalten) entfernen, um Duplikate zu vermeiden
    shared_cols = [col for col in krs_df.columns if col in zensus_df.columns and col != 'Kennziffer']
    zensus_clean = zensus_df.drop(columns=shared_cols)

    # Merge über AGS (KRS) und Kennziffer (Zensus)
    merged_df = pd.merge(
        zensus_clean,
        krs_df,
        left_on='Kennziffer',
        right_on='AGS',
        how='left',
        validate='one_to_many'
    )

    # 'Kennziffer' entfernen und Spaltenreihenfolge anpassen
    merged_df.drop(columns='Kennziffer', inplace=True)
    krs_cols = krs_df.columns.tolist()
    other_cols = [col for col in merged_df.columns if col not in krs_cols]
    merged_df = merged_df[krs_cols + other_cols]
    
    return merged_df

# Pfad zum KRS-Übersicht einlesen
krs_path = r"C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\Verwaltungsgebiete\KRS_Uebersicht.csv"
krs_df = pd.read_csv(
    krs_path,
    sep=',',
    encoding='utf-8',
    quotechar='"',
    dtype={'AGS': str, 'ARS': str}
)

# Ordner mit den Zensus-Dateien
zensus_dir = r"C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\INKAR"
output_dir = r"C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\INKAR\processed_files"


# Alle CSV-Dateien im INKAR-Ordner durchgehen und bereinigen
for f in glob(os.path.join(zensus_dir, "*.csv")):
    try:
        # Zensus-Daten bereinigen
        df_cleaned = clean_zensus(f, krs_df)
        
        # Neuen Dateinamen erstellen
        cleaned_filename = os.path.splitext(os.path.basename(f))[0] +  "_cleaned.csv"
        cleaned_path = os.path.join(output_dir, cleaned_filename)
        
        # Bereinigte Datei speichern
        df_cleaned.to_csv(cleaned_path, index=False)
        print(f"✓ Datei {os.path.basename(f)} erfolgreich bereinigt und gespeichert als {cleaned_filename}")
    
    except Exception as e:
        print(f"✗ Fehler bei {os.path.basename(f)} – {e}")


✓ Datei Absolutzahlen.csv erfolgreich bereinigt und gespeichert als Absolutzahlen_cleaned.csv
✓ Datei Beschaeftigung_Qualifikation.csv erfolgreich bereinigt und gespeichert als Beschaeftigung_Qualifikation_cleaned.csv
✓ Datei Beschaeftigung_Struktur.csv erfolgreich bereinigt und gespeichert als Beschaeftigung_Struktur_cleaned.csv
✓ Datei Beschaeftigung_Wirtschafts_u_Berufszweige.csv erfolgreich bereinigt und gespeichert als Beschaeftigung_Wirtschafts_u_Berufszweige_cleaned.csv
✓ Datei Bevoelkerungsstruktur.csv erfolgreich bereinigt und gespeichert als Bevoelkerungsstruktur_cleaned.csv
✓ Datei Flaechennutzung.csv erfolgreich bereinigt und gespeichert als Flaechennutzung_cleaned.csv
✓ Datei Leistungsempfaenger.csv erfolgreich bereinigt und gespeichert als Leistungsempfaenger_cleaned.csv
✓ Datei Medizinische_Versorgung.csv erfolgreich bereinigt und gespeichert als Medizinische_Versorgung_cleaned.csv
✓ Datei SDG_Indikatoren.csv erfolgreich bereinigt und gespeichert als SDG_Indikatoren_clea

In [95]:
import pandas as pd
from functools import reduce
import os
from glob import glob

# 1) Verzeichnis mit den _cleaned.csv  
zensus_dir = r"C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\INKAR\processed_files"
output_dir = zensus_dir

# 2) Alle _cleaned.csv finden
cleaned_files = glob(os.path.join(zensus_dir, "*_cleaned.csv"))

# 3) Schlüsselspalten, auf denen gemerged wird
keys = ['AGS', 'ARS', 'GF', 'GEN', 'BEZ']

# 4) Alle Dateien einlesen – mit dtype für alle Keys = str
dfs = []
for fn in cleaned_files:
    df = pd.read_csv(
        fn,
        # falls deine Dateien mit Semikolon getrennt sind, sep=';' einstellen
        sep=',',
        dtype={k: str for k in keys},
        encoding='utf-8',
        quotechar='"'
    )
    dfs.append(df)

# 5) Alle DFS schrittweise über die Keys outer-joinen
combined = reduce(
    lambda left, right: pd.merge(left, right, on=keys, how='outer'),
    dfs
)

# 6) Doppelte Spalten (falls welche durch Merge mit identischem Namen entstehen) entfernen
combined = combined.loc[:, ~combined.columns.duplicated()]

# 7) Speichern
output_path = os.path.join(output_dir, "combined_cleaned_zensus.csv")
combined.to_csv(output_path, index=False)

print(f"✅ Fertig! Datei gespeichert als {output_path}")


✅ Fertig! Datei gespeichert als C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\INKAR\processed_files\combined_cleaned_zensus.csv
